<a href="https://colab.research.google.com/github/neohack22/IASD/blob/DeepLearning/Multiple_Inputs_and_Outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example

## model in a few lines with the Functional API

In [0]:
# Number of unique issue tags
num_tags = 12

# Size of vocabulary obtained when preprocessing text data
num_words = 10000

# Number of departments for predictions
num_departments = 4

In [0]:
# Variable_length sequence of ints
title_input = keras.Input(shape=(None,), name='title')

# Variable-length sequence of ints
body_input = keras.Input(shape=(None,) name='body')

# Binary vectors of size `num_tags`
tags_input = keras.Input(shape=(num_tags,), name='tags')

In [0]:
# Embed each word in the title into a 64-dimensional vector
title_features = layers.Embedding(num_words, 64)(title_input)

# Embed each word in the text into a 64-dimensional vector
body_features = layers.Embedding(num_words, 64)(body_input)

In [0]:
""" Reduce sequence of embedded words in the title into 
a single 128-dimensional vector"""
title_features = layers.LSTM(128)(title_features)

""" Reduce sequence of embedded words in the body 
into a single 32_dimensional vector """
body_features = layers.LSTM(32)(body_features)

In [0]:
""" Merge all available features 
into a single large vector via concatenation """"
x = layers.concatenate([title_features, body_features, tags_input])

In [0]:
""" Stick a logistic regression for priority prediction 
on top of the features """"
priority_pred = \
layers.Dense(1, activation='sigmoid', name='priority')(x)

# Stick a department classifier on top of the features
department_pred = \
layers.Dense(num_departments, activation='softmax', \
             name='department')(x)

In [0]:
# Instantiate an end-to-end model predicting both priority and department
model = keras.Model(inputs=[title_input, body_input, tags_input],
                    outputs=[priority_pred, department_pred])

In [0]:
keras.utils.plot_model(model, 'multi_input_and_output_model.png', 
                       show_shapes=True)

## compiling

In [0]:
"""
# iso
model.compile(optimizer=keras.optyimizers.RMSprop(1e-3),
              loss=['binary_crossentropy', 'categorical_crossentropy'],
              loss_weights=[1., 0.2])
"""

In [0]:
# specify the loss

model.compile(optimizer=keras.optimizers.RMSprop(1e-3),
              loss={'priority': 'binary_crossentropy',
                    'department': 'categorical_crossentropty'},
              loss_weights=[1., 0.2])

## train the model

In [0]:
import numpy as np

# Dummy input data
title_data = np.random.randint(num_words, size=(1280,10))
body_data = np.random.randint(num_words, size=(1280, 100))
tags_data = np.random.randint(2, size=(1280, num_tags)).astype('float32')

# Dummy target data
priority_targets = np.random.random(size=(1280, 1))
dept_targets = np.random.randint(2, size=(1280, num_departments))

model.fit({'title': title_data, 'body': body_data, 'tags': tags_data},
          {'priority': priority_targets, 'department': dept_targets},
          epochs=2,
          batch_size=32)